[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_ml_features/EMQA_ml_features.ipynb)

# EMQA_ml_features

Feature engineering for electricity price prediction using Romanian and German market data.
Demonstrates creation of lag features, rolling statistics, temporal encodings, and cross-market features.

**Output:** `ml_features.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Professional plot styling - transparent background, no grid
plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
# Load Romanian and German electricity price data
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_actual_vs_predicted/ro_de_prices_full.csv'
df = pd.read_csv(url, parse_dates=['date'], index_col='date')
print(f'Loaded {len(df)} observations from {df.index.min().date()} to {df.index.max().date()}')
print(f'Columns: {df.columns.tolist()}')
df.head()

## 1. Lag Features

Lag features capture autocorrelation in the time series. For electricity prices, yesterday's price
is often the best predictor of today's price. We create lags at various horizons:
- **Short-term**: 1-7 days (recent price memory)
- **Weekly**: 7 days (same day last week)
- **Monthly**: 30 days (seasonal patterns)

In [ ]:
# Create lag features for Romanian prices
data = df[['ro_price']].copy()

# Short-term lags (1-5 days)
for lag in [1, 2, 3, 4, 5]:
    data[f'ro_lag_{lag}'] = data['ro_price'].shift(lag)

# Weekly and monthly lags
data['ro_lag_7'] = data['ro_price'].shift(7)    # Same day last week
data['ro_lag_14'] = data['ro_price'].shift(14)  # Two weeks ago
data['ro_lag_30'] = data['ro_price'].shift(30)  # One month ago

print("Lag features created:")
print([c for c in data.columns if 'lag' in c])

In [ ]:
# Visualize lag correlations
lags_to_plot = [1, 2, 3, 7, 14, 30]
correlations = [data['ro_price'].corr(data[f'ro_lag_{lag}']) for lag in lags_to_plot]

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(range(len(lags_to_plot)), correlations, color=COLORS['blue'], alpha=0.8, edgecolor='white')
ax.set_xticks(range(len(lags_to_plot)))
ax.set_xticklabels([f'Lag {l}' for l in lags_to_plot])
ax.set_ylabel('Correlation with Current Price')
ax.set_title('Lag Feature Correlations with Romanian Electricity Price', fontsize=14, fontweight='bold')

# Add value labels
for bar, corr in zip(bars, correlations):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{corr:.3f}', ha='center', va='bottom', fontsize=10)

ax.set_ylim(0, 1.1)
plt.tight_layout()
plt.show()

## 2. Rolling Statistics

Rolling statistics capture trends and volatility regimes:
- **Rolling Mean**: Trend indicator (is price above/below recent average?)
- **Rolling Std**: Volatility regime (calm vs turbulent markets)
- **Rolling Min/Max**: Support/resistance levels

In [ ]:
# Create rolling statistics (shift by 1 to avoid look-ahead bias)
windows = [7, 14, 30]

for w in windows:
    # Rolling mean (shifted to avoid look-ahead)
    data[f'ro_ma_{w}'] = data['ro_price'].shift(1).rolling(w).mean()
    
    # Rolling standard deviation (volatility)
    data[f'ro_std_{w}'] = data['ro_price'].shift(1).rolling(w).std()
    
    # Rolling min and max (support/resistance)
    data[f'ro_min_{w}'] = data['ro_price'].shift(1).rolling(w).min()
    data[f'ro_max_{w}'] = data['ro_price'].shift(1).rolling(w).max()

# Relative position within rolling range (0 = at min, 1 = at max)
for w in windows:
    range_w = data[f'ro_max_{w}'] - data[f'ro_min_{w}']
    data[f'ro_position_{w}'] = (data['ro_price'].shift(1) - data[f'ro_min_{w}']) / range_w.replace(0, np.nan)

print("Rolling statistics created:")
print([c for c in data.columns if any(x in c for x in ['ma_', 'std_', 'min_', 'max_', 'position_'])])

In [ ]:
# Visualize rolling features
plot_data = data.dropna().iloc[-365:]  # Last year

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Panel 1: Price with rolling means
ax1 = axes[0]
ax1.plot(plot_data.index, plot_data['ro_price'], color=COLORS['blue'], lw=1.5, label='Price')
ax1.plot(plot_data.index, plot_data['ro_ma_7'], color=COLORS['orange'], lw=1.5, ls='--', label='MA(7)')
ax1.plot(plot_data.index, plot_data['ro_ma_30'], color=COLORS['red'], lw=1.5, ls='--', label='MA(30)')
ax1.set_ylabel('Price (EUR/MWh)')
ax1.set_title('Rolling Mean Features', fontsize=13, fontweight='bold')
ax1.legend(loc='upper right', frameon=False)

# Panel 2: Rolling volatility
ax2 = axes[1]
ax2.fill_between(plot_data.index, plot_data['ro_std_7'], color=COLORS['purple'], alpha=0.3, label='Std(7)')
ax2.plot(plot_data.index, plot_data['ro_std_30'], color=COLORS['red'], lw=1.5, label='Std(30)')
ax2.set_ylabel('Volatility (EUR/MWh)')
ax2.set_title('Rolling Volatility Features', fontsize=13, fontweight='bold')
ax2.legend(loc='upper right', frameon=False)

# Panel 3: Position within range
ax3 = axes[2]
ax3.fill_between(plot_data.index, 0, plot_data['ro_position_30'], 
                 where=plot_data['ro_position_30'] >= 0.5, color=COLORS['green'], alpha=0.5, label='Upper half')
ax3.fill_between(plot_data.index, 0, plot_data['ro_position_30'], 
                 where=plot_data['ro_position_30'] < 0.5, color=COLORS['red'], alpha=0.5, label='Lower half')
ax3.axhline(0.5, color=COLORS['gray'], ls='--', lw=1)
ax3.set_ylabel('Position (0-1)')
ax3.set_xlabel('Date')
ax3.set_title('Position Within 30-Day Range', fontsize=13, fontweight='bold')
ax3.legend(loc='upper right', frameon=False)
ax3.set_ylim(0, 1)

fig.tight_layout()
plt.show()

## 3. Temporal Features

Electricity prices exhibit strong temporal patterns:
- **Day of week**: Weekends typically have lower demand/prices
- **Month**: Seasonal heating/cooling demand
- **Weekend flag**: Binary indicator for weekend days

In [ ]:
# Create temporal features
data['day_of_week'] = data.index.dayofweek      # 0=Monday, 6=Sunday
data['month'] = data.index.month                 # 1-12
data['day_of_month'] = data.index.day            # 1-31
data['week_of_year'] = data.index.isocalendar().week.astype(int)
data['is_weekend'] = (data.index.dayofweek >= 5).astype(int)

# Cyclical encoding for day of week and month (sine/cosine transformation)
# This helps models understand that December is close to January
data['dow_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
data['dow_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

print("Temporal features created:")
print([c for c in data.columns if c in ['day_of_week', 'month', 'day_of_month', 'week_of_year', 
                                         'is_weekend', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']])

In [ ]:
# Visualize temporal patterns
plot_df = data.dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Day of week pattern
ax1 = axes[0]
dow_means = plot_df.groupby('day_of_week')['ro_price'].mean()
dow_std = plot_df.groupby('day_of_week')['ro_price'].std()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

colors_dow = [COLORS['red'] if i >= 5 else COLORS['blue'] for i in range(7)]
bars = ax1.bar(range(7), dow_means.values, yerr=dow_std.values, 
               color=colors_dow, alpha=0.8, edgecolor='white', capsize=3)
ax1.set_xticks(range(7))
ax1.set_xticklabels(days)
ax1.set_ylabel('Average Price (EUR/MWh)')
ax1.set_title('Average Price by Day of Week', fontsize=13, fontweight='bold')

# Add legend
import matplotlib.patches as mpatches
legend_items = [
    mpatches.Patch(color=COLORS['blue'], label='Weekday'),
    mpatches.Patch(color=COLORS['red'], label='Weekend')
]
ax1.legend(handles=legend_items, loc='upper right', frameon=False)

# Monthly pattern
ax2 = axes[1]
month_means = plot_df.groupby('month')['ro_price'].mean()
month_std = plot_df.groupby('month')['ro_price'].std()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Color by season
season_colors = [COLORS['blue'], COLORS['blue'], COLORS['green'], COLORS['green'], COLORS['green'],
                 COLORS['orange'], COLORS['orange'], COLORS['orange'], COLORS['amber'],
                 COLORS['amber'], COLORS['amber'], COLORS['blue']]

bars = ax2.bar(range(12), month_means.values, yerr=month_std.values,
               color=season_colors, alpha=0.8, edgecolor='white', capsize=2)
ax2.set_xticks(range(12))
ax2.set_xticklabels(months, rotation=45)
ax2.set_ylabel('Average Price (EUR/MWh)')
ax2.set_title('Average Price by Month', fontsize=13, fontweight='bold')

# Season legend
season_legend = [
    mpatches.Patch(color=COLORS['blue'], label='Winter'),
    mpatches.Patch(color=COLORS['green'], label='Spring'),
    mpatches.Patch(color=COLORS['orange'], label='Summer'),
    mpatches.Patch(color=COLORS['amber'], label='Autumn')
]
ax2.legend(handles=season_legend, loc='upper right', frameon=False, ncol=2)

fig.tight_layout()
plt.show()

## 4. Cross-Market Features

Romanian and German electricity markets are interconnected. German prices can provide
valuable predictive information for Romanian prices:
- **German price lags**: DE market often leads RO
- **Price spread**: RO-DE spread captures import/export dynamics
- **Spread momentum**: Changes in the spread signal market shifts

In [ ]:
# Add German price features if available
if 'de_price' in df.columns:
    data['de_price'] = df['de_price']
    
    # German price lags
    for lag in [1, 2, 7]:
        data[f'de_lag_{lag}'] = data['de_price'].shift(lag)
    
    # Price spread (RO - DE)
    data['spread'] = data['ro_price'] - data['de_price']
    data['spread_lag_1'] = data['spread'].shift(1)
    
    # Spread momentum (change in spread)
    data['spread_change'] = data['spread'].diff()
    data['spread_ma_7'] = data['spread'].shift(1).rolling(7).mean()
    
    # Correlation-based features
    data['de_ro_ratio'] = data['de_price'].shift(1) / data['ro_price'].shift(1)
    
    print("Cross-market features created:")
    print([c for c in data.columns if 'de_' in c or 'spread' in c or 'ratio' in c])
else:
    print("German price data not available - skipping cross-market features")

In [ ]:
# Visualize cross-market relationships
if 'de_price' in data.columns:
    plot_df = data.dropna().iloc[-365:]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Panel 1: RO vs DE prices
    ax1 = axes[0, 0]
    ax1.plot(plot_df.index, plot_df['ro_price'], color=COLORS['blue'], lw=1.5, label='Romania')
    ax1.plot(plot_df.index, plot_df['de_price'], color=COLORS['green'], lw=1.5, alpha=0.7, label='Germany')
    ax1.set_ylabel('Price (EUR/MWh)')
    ax1.set_title('Romanian vs German Electricity Prices', fontsize=13, fontweight='bold')
    ax1.legend(loc='upper right', frameon=False)
    
    # Panel 2: Price spread
    ax2 = axes[0, 1]
    ax2.fill_between(plot_df.index, plot_df['spread'], 0,
                     where=plot_df['spread'] >= 0, color=COLORS['green'], alpha=0.5, label='RO > DE')
    ax2.fill_between(plot_df.index, plot_df['spread'], 0,
                     where=plot_df['spread'] < 0, color=COLORS['red'], alpha=0.5, label='RO < DE')
    ax2.axhline(0, color=COLORS['gray'], ls='--', lw=1)
    ax2.set_ylabel('Spread (EUR/MWh)')
    ax2.set_title('RO-DE Price Spread', fontsize=13, fontweight='bold')
    ax2.legend(loc='upper right', frameon=False)
    
    # Panel 3: Scatter plot RO vs DE
    ax3 = axes[1, 0]
    ax3.scatter(data['de_price'], data['ro_price'], alpha=0.3, s=10, color=COLORS['blue'])
    
    # Regression line
    valid = data[['de_price', 'ro_price']].dropna()
    z = np.polyfit(valid['de_price'], valid['ro_price'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(valid['de_price'].min(), valid['de_price'].max(), 100)
    ax3.plot(x_line, p(x_line), color=COLORS['red'], lw=2, label=f'y = {z[0]:.2f}x + {z[1]:.1f}')
    
    corr = valid['de_price'].corr(valid['ro_price'])
    ax3.text(0.05, 0.95, f'Correlation: {corr:.3f}', transform=ax3.transAxes, 
             fontsize=11, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax3.set_xlabel('German Price (EUR/MWh)')
    ax3.set_ylabel('Romanian Price (EUR/MWh)')
    ax3.set_title('RO vs DE Price Correlation', fontsize=13, fontweight='bold')
    ax3.legend(loc='lower right', frameon=False)
    
    # Panel 4: Lead-lag correlation
    ax4 = axes[1, 1]
    lags = range(-7, 8)
    correlations = [data['ro_price'].corr(data['de_price'].shift(lag)) for lag in lags]
    
    colors_lag = [COLORS['green'] if c == max(correlations) else COLORS['blue'] for c in correlations]
    ax4.bar(lags, correlations, color=colors_lag, alpha=0.8, edgecolor='white')
    ax4.axvline(0, color=COLORS['gray'], ls='--', lw=1)
    ax4.set_xlabel('Lag (days, negative = DE leads)')
    ax4.set_ylabel('Correlation')
    ax4.set_title('Lead-Lag Correlation: RO vs DE', fontsize=13, fontweight='bold')
    
    fig.tight_layout()
    plt.show()

## 5. Feature Summary

Compile all engineered features and display correlations with the target variable.

In [ ]:
# Final feature dataset
data['target'] = data['ro_price']  # Target: current price (for demonstration)

# Exclude non-feature columns
exclude_cols = ['ro_price', 'de_price', 'target', 'spread']
feature_cols = [c for c in data.columns if c not in exclude_cols]

# Calculate correlations with target
clean_data = data.dropna()
correlations = clean_data[feature_cols].corrwith(clean_data['target']).sort_values(ascending=False)

print(f"Total features engineered: {len(feature_cols)}")
print(f"Dataset size after dropping NaN: {len(clean_data)} observations")
print("\nTop 15 features by correlation with target:")
print(correlations.head(15).to_string())

In [ ]:
# Visualize top feature correlations
top_features = correlations.abs().sort_values(ascending=True).tail(15)

fig, ax = plt.subplots(figsize=(12, 8))

# Color by feature type
def get_feature_color(name):
    if 'lag' in name:
        return COLORS['blue']
    elif 'ma_' in name or 'std_' in name or 'min_' in name or 'max_' in name:
        return COLORS['green']
    elif 'de_' in name or 'spread' in name:
        return COLORS['purple']
    elif name in ['day_of_week', 'month', 'is_weekend', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']:
        return COLORS['orange']
    else:
        return COLORS['gray']

bar_colors = [get_feature_color(f) for f in top_features.index]
bars = ax.barh(range(len(top_features)), top_features.values, color=bar_colors, alpha=0.85, edgecolor='white')

ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features.index, fontsize=10)
ax.set_xlabel('Absolute Correlation with Target', fontsize=12)
ax.set_title('Top 15 Features by Correlation with Price', fontsize=15, fontweight='bold')

# Value labels
for i, (val, bar) in enumerate(zip(top_features.values, bars)):
    ax.text(val + 0.005, i, f'{val:.3f}', va='center', fontsize=9)

# Legend
import matplotlib.patches as mpatches
legend_items = [
    mpatches.Patch(color=COLORS['blue'], label='Lag features'),
    mpatches.Patch(color=COLORS['green'], label='Rolling statistics'),
    mpatches.Patch(color=COLORS['purple'], label='Cross-market features'),
    mpatches.Patch(color=COLORS['orange'], label='Temporal features'),
]
ax.legend(handles=legend_items, loc='upper center', bbox_to_anchor=(0.5, -0.08),
          frameon=False, ncol=4, fontsize=10)

fig.tight_layout()
save_fig(fig, 'ml_features.pdf')
plt.show()

In [ ]:
# Summary table of feature categories
feature_categories = {
    'Lag Features': [c for c in feature_cols if 'lag' in c],
    'Rolling Statistics': [c for c in feature_cols if any(x in c for x in ['ma_', 'std_', 'min_', 'max_', 'position_'])],
    'Temporal Features': [c for c in feature_cols if c in ['day_of_week', 'month', 'day_of_month', 'week_of_year', 
                                                            'is_weekend', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']],
    'Cross-Market': [c for c in feature_cols if 'de_' in c or 'spread' in c or 'ratio' in c]
}

print("\n" + "="*60)
print("FEATURE ENGINEERING SUMMARY")
print("="*60)
for category, features in feature_categories.items():
    print(f"\n{category} ({len(features)} features):")
    for f in features:
        corr = correlations.get(f, 0)
        print(f"  - {f}: corr = {corr:.3f}")